<h1 align="center">Computación de Alto Desempeño</h1>
<h1 align="center">Diseño de Programas Paralelos</h1>
<h1 align="center">2024</h1>
<h1 align="center">MEDELLÍN - COLOMBIA </h1>

*** 
|[![Outlook](https://img.shields.io/badge/Microsoft_Outlook-0078D4?style=plastic&logo=microsoft-outlook&logoColor=white)](mailto:calvarezh@udemedellin.edu.co)||[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/carlosalvarezh/HPC/blob/main/HPC03_DisenoProgramasParalelo.ipynb)
|-:|:-|--:|
|[![LinkedIn](https://img.shields.io/badge/linkedin-%230077B5.svg?style=plastic&logo=linkedin&logoColor=white)](https://www.linkedin.com/in/carlosalvarez5/)|[![@alvarezhenao](https://img.shields.io/twitter/url/https/twitter.com/alvarezhenao.svg?style=social&label=Follow%20%40alvarezhenao)](https://twitter.com/alvarezhenao)|[![@carlosalvarezh](https://img.shields.io/badge/github-%23121011.svg?style=plastic&logo=github&logoColor=white)](https://github.com/carlosalvarezh)|

<table>
 <tr align=left><td><img align=left src="https://github.com/carlosalvarezh/Curso_CEC_EAFIT/blob/main/images/CCLogoColorPop1.gif?raw=true" width="25">
 <td>Text provided under a Creative Commons Attribution license, CC-BY. All code is made available under the FSF-approved MIT license.(c) Carlos Alberto Alvarez Henao</td>
</table>

***

# Aviso Legal sobre estas Notas de Clase

Las presentes notas de clase han sido elaboradas con fines educativos y como material de apoyo para el aprendizaje y comprensión de la computación paralela. Estas notas están basadas en información y contenidos derivados del tutorial *["Introduction to Parallel Computing Tutorial"](https://hpc.llnl.gov/documentation/tutorials/introduction-parallel-computing-tutorial)* disponible en el sitio web del *Laboratorio Nacional Lawrence Livermore* (https://hpc.llnl.gov/). Este material se proporciona tal cual, sin garantías de exactitud completa o de la aplicabilidad para un fin particular.

A pesar de que se ha hecho un esfuerzo por asegurar la precisión y utilidad de estas notas, los usuarios deben tener en cuenta que los conceptos, aplicaciones y técnicas de la computación paralela están en constante evolución, y se recomienda consultar múltiples fuentes y la documentación oficial más actual para obtener la información más reciente y completa.

Por favor, considere que cualquier ejemplo, referencia o cita de "Introducción a la Computación Paralela" se proporciona con el objetivo de ilustrar los conceptos y principios básicos de la computación paralela y no debe considerarse como una guía exhaustiva o definitiva sobre el tema.
***

<p float="center">
  <img src="https://github.com/carlosalvarezh/EstructuraDatosAlgoritmos2/blob/main/images/Parallel00.jpg?raw=true" width="500" />
</p>

## Diseño de Programas Paralelos

### Paralelización Automática vs. Manual

El diseño y desarrollo de programas paralelos ha sido tradicionalmente un proceso muy manual. Generalmente, el programador es responsable tanto de identificar como de implementar el paralelismo.

Desarrollar códigos paralelos manualmente es a menudo un proceso que consume tiempo, es complejo, propenso a errores y requiere iteraciones.

Durante varios años, han estado disponibles diversas herramientas para asistir al programador en la conversión de programas seriales a programas paralelos. El tipo de herramienta más común utilizado para paralelizar automáticamente un programa serial es un compilador paralelizante o un preprocesador.

Un compilador paralelizante generalmente funciona de dos maneras diferentes:


***Completamente Automático***
- El compilador analiza el código fuente e identifica oportunidades para el paralelismo.
- El análisis incluye la identificación de inhibidores del paralelismo y posiblemente una ponderación de costos sobre si el paralelismo realmente mejoraría el rendimiento.
- Los bucles (do, for) son el objetivo más frecuente para la paralelización automática.

***Dirigido por el Programador***
- Utilizando "directivas de compilador" o posiblemente banderas de compilador, el programador le indica explícitamente al compilador cómo paralelizar el código.
- Puede ser usado en conjunto con algún grado de paralelización automática también.

La paralelización generada por el compilador más común se realiza usando memoria compartida en el nodo y hilos (como OpenMP).

Si estás comenzando con un código serial existente y tienes restricciones de tiempo o presupuesto, entonces la paralelización automática puede ser la respuesta. Sin embargo, hay varias advertencias importantes que se aplican a la paralelización automática:

- Puede producir resultados incorrectos.
- El rendimiento puede degradarse.
- Es mucho menos flexible que la paralelización manual.
- Limitada a un subconjunto (principalmente bucles) del código.
- Puede que no paralelice el código si el análisis del compilador sugiere que hay inhibidores o el código es demasiado complejo.

El resto de esta sección se aplica al método manual de desarrollo de códigos paralelos.

### Comprender el Problema y el Programa

$$\text{Programas = algoritmos + datos + (hardware)}$$

Sin duda, el primer paso en el desarrollo de software paralelo es primero comprender el problema que deseas resolver en paralelo. Si estás comenzando con un programa serial, esto significa entender también el código existente.
Antes de dedicar tiempo en un intento de desarrollar una solución paralela para un problema, determina si el problema es uno que realmente se pueda paralelizar.

- **Ejemplo de un problema fácil de paralelizar:** Calcular la energía potencial para cada una de varias miles de conformaciones independientes de una molécula. Una vez hecho esto, encontrar la conformación de energía mínima. Este problema puede resolverse en paralelo. Cada una de las conformaciones moleculares es determinable independientemente. El cálculo de la conformación de energía mínima también es un problema paralelizable.

- **Ejemplo de un problema y algoritmo con poco o ningún paralelismo:** Cálculo de los primeros $10,000$ miembros de la serie de Fibonacci $(0,1,1,2,3,5,8,13,21, \ldots)$ mediante el uso de la fórmula:

$$F(n) = F(n-1) + F(n-2)$$

El cálculo del valor de $F(n)$ utiliza los de $F(n-1)$ y $F(n-2)$, que deben ser calculados primero.

Un ejemplo de un algoritmo paralelo para resolver este problema (usando la fórmula de Binet):

$$F_n = \frac{\varphi^n - (-\varphi)^{-n}}{\sqrt{5}} = \frac{\varphi^n - (-\varphi)^{-n}}{2\varphi - 1} $$

  donde

$$\varphi = \frac{1 + \sqrt{5}}{2} \approx 1.6180339887 \ldots $$

- **Identificar los puntos críticos del programa:**  
  - Conoce dónde se está realizando la mayor parte del trabajo real. La mayoría de los programas científicos y técnicos generalmente realizan la mayor parte de su trabajo en pocos lugares.
  - Los perfiles y herramientas de análisis de rendimiento pueden ayudar aquí.
  - Concéntrate en paralelizar los puntos críticos e ignora aquellas secciones del programa que representan poco uso de CPU.

- **Identificar cuellos de botella en el programa:**  
  - ¿Hay áreas que son desproporcionadamente lentas o causan que el trabajo paralelizable se detenga o se posponga? Por ejemplo, las operaciones de I/O generalmente ralentizan un programa.
  - Puede ser posible reestructurar el programa o usar un algoritmo diferente para reducir o eliminar áreas lentas innecesarias.

- **Identificar inhibidores al paralelismo.** Una clase común de inhibidor es la dependencia de datos, como se demostró con la secuencia de Fibonacci anterior.  
  - Investiga otros algoritmos si es posible. Esto puede ser la consideración más importante al diseñar una aplicación paralela.
  - Aprovecha el software paralelo de terceros optimizado y las bibliotecas matemáticas altamente optimizadas disponibles de proveedores líderes (ESSL de IBM, MKL de Intel, AMCL de AMD, etc.).

### Particionamiento en Programación Paralela

El particionamiento es uno de los primeros pasos en el diseño de un programa paralelo y consiste en dividir el problema en "trozos" discretos de trabajo que pueden ser distribuidos a múltiples tareas. Este proceso es conocido como descomposición o particionamiento y es fundamental para la eficiencia y efectividad de las soluciones paralelas. Existen dos formas básicas de particionar el trabajo computacional entre las tareas paralelas: la descomposición de dominio y la descomposición funcional.

#### Descomposición de Dominio

<p float="center">
  <img src="https://github.com/carlosalvarezh/EstructuraDatosAlgoritmos2/blob/main/images/Parallel36.gif?raw=true" width="350" />
</p>

En este tipo de particionamiento, los datos asociados con un problema son descompuestos. Cada tarea paralela entonces trabaja en una porción de los datos. 

**Maneras de particionar datos:**

<p float="center">
  <img src="https://github.com/carlosalvarezh/EstructuraDatosAlgoritmos2/blob/main/images/Parallel37.gif?raw=true" width="350" />
</p>

- **Por Bloques:** Los datos se dividen en bloques contiguos, cada uno asignado a una tarea diferente. Esto es común en problemas donde los datos se pueden segmentar naturalmente, como en simulaciones espaciales o temporales.
  
- **Por Ciclos:** Los datos se distribuyen en un patrón cíclico entre las tareas, lo que puede ayudar a balancear la carga de trabajo cuando las operaciones realizadas sobre los datos varían en complejidad.
  
- **Por Dispersión:** Los datos se dividen de manera que cada tarea recibe fragmentos que están dispersos a través del conjunto de datos completo. Esto puede ser útil en situaciones donde la interacción entre elementos de datos es compleja y no se limita a áreas contiguas.

La descomposición de dominio es especialmente útil en problemas que se basan en la manipulación de grandes volúmenes de datos, como en el procesamiento de imágenes, simulaciones de fluidos, y análisis de grandes conjuntos de datos.



#### Descomposición Funcional

En este enfoque, el foco está en la computación que debe ser realizada más que en los datos manipulados por la computación. El problema se descompone según el trabajo que debe realizarse. Cada tarea realiza entonces una porción del trabajo general.

<p float="center">
  <img src="https://github.com/carlosalvarezh/EstructuraDatosAlgoritmos2/blob/main/images/Parallel38.gif?raw=true" width="350" />
</p>

**Características de la descomposición funcional:**
- **División por Tareas:** El problema se divide en diferentes funciones o tareas específicas, cada una realizada por diferentes procesos o hilos.
  
- **Especialización:** Cada tarea paralela se especializa en una función particular, lo cual puede llevar a una mayor eficiencia en tareas que requieren habilidades o recursos específicos.
  
- **Independencia:** Las tareas son a menudo independientes o tienen interdependencias mínimas, lo que reduce la necesidad de sincronización y comunicación entre tareas.

La descomposición funcional se presta bien a problemas que pueden ser divididos en diferentes tareas que son relativamente independientes y pueden ser ejecutadas concurrentemente, como en sistemas de manejo de transacciones, donde diferentes tareas pueden manejar diferentes aspectos de un proceso de negocio.

El éxito del particionamiento en programación paralela depende en gran medida de la naturaleza del problema y de cómo se descomponen los datos o las funciones. Una buena estrategia de particionamiento maximiza la eficiencia del paralelismo minimizando la comunicación necesaria entre tareas y equilibrando la carga de trabajo entre los procesadores. Identificar el tipo adecuado de particionamiento es crucial y puede variar significativamente de un problema a otro, influenciado por los objetivos de rendimiento y las características específicas del sistema computacional utilizado.

### Comunicaciones en Programación Paralela

#### ¿Quién necesita comunicaciones?

La necesidad de comunicación entre tareas depende del problema:

**No Necesitas Comunicaciones**

<p float="center">
  <img src="https://github.com/carlosalvarezh/EstructuraDatosAlgoritmos2/blob/main/images/Parallel42.gif?raw=true" width="200" />
</p>

- Algunos tipos de problemas pueden descomponerse y ejecutarse en paralelo sin apenas necesidad de que las tareas compartan datos. A estos problemas se les suele llamar paralelos embarazosamente - requieren poca o ninguna comunicación.
- **Ejemplo:** Imagina una operación de procesamiento de imagen donde cada píxel en una imagen en blanco y negro necesita invertir su color. Los datos de la imagen se pueden distribuir fácilmente a múltiples tareas que actúan independientemente unas de otras para realizar su parte del trabajo.

**Necesitas Comunicaciones**

<p float="center">
  <img src="https://github.com/carlosalvarezh/EstructuraDatosAlgoritmos2/blob/main/images/Parallel43.gif?raw=true" width="200" />
</p>

- La mayoría de las aplicaciones paralelas no son tan simples y sí requieren que las tareas compartan datos entre sí.
- **Ejemplo:** Un problema de difusión de calor en 2-D requiere que una tarea conozca las temperaturas calculadas por las tareas que tienen datos adyacentes. Los cambios en los datos vecinos tienen un efecto directo en los datos de la tarea.


#### Factores a Considerar

Hay varios factores importantes a considerar al diseñar las comunicaciones inter-tareas de tu programa:

**Sobrecarga de Comunicación (overhead)**

<p float="center">
  <img src="https://github.com/carlosalvarezh/EstructuraDatosAlgoritmos2/blob/main/images/Parallel44.jpeg?raw=true" width="350" />
</p>

- La comunicación inter-tareas casi siempre implica sobrecarga.
- Ciclos de máquina y recursos que podrían usarse para la computación se usan en cambio para empaquetar y transmitir datos.
- Las comunicaciones frecuentemente requieren algún tipo de sincronización entre tareas, lo que puede resultar en que las tareas pasen tiempo "esperando" en lugar de trabajar.
- El tráfico de comunicación competidor puede saturar el ancho de banda de red disponible, agravando aún más los problemas de rendimiento.

**Latencia vs. Ancho de Banda**
- **Latencia:** Es el tiempo que tarda en enviarse un mensaje mínimo (de 0 bytes) de un punto A a un punto B. Comúnmente expresada en microsegundos.
- **Ancho de Banda:** Es la cantidad de datos que se pueden comunicar por unidad de tiempo. Comúnmente expresado en megabytes/seg o gigabytes/seg.
- Enviar muchos mensajes pequeños puede causar que la latencia domine las sobrecargas de comunicación. A menudo es más eficiente empaquetar mensajes pequeños en un mensaje más grande, aumentando así el ancho de banda efectivo de las comunicaciones.

**Visibilidad de las Comunicaciones**
- En el **Modelo de Paso de Mensajes**, las comunicaciones son explícitas y generalmente bastante visibles y bajo el control del programador.
- En el **Modelo de Paralelismo de Datos**, las comunicaciones a menudo ocurren de manera transparente para el programador, especialmente en arquitecturas de memoria distribuida. El programador puede no saber exactamente cómo se están realizando las comunicaciones inter-tareas.

**Comunicaciones Sincrónicas vs. Asincrónicas**
- **Sincrónicas:** Requieren algún tipo de "apretón de manos" entre tareas que comparten datos. Esto puede estar explícitamente estructurado en el código por el programador.
- **Asincrónicas:** Permiten que las tareas transfieran datos independientemente una de otra. Por ejemplo, la tarea 1 puede preparar y enviar un mensaje a la tarea 2 y luego comenzar inmediatamente a realizar otro trabajo.

**Ámbito de las Comunicaciones**

- Saber qué tareas deben comunicarse entre sí es crítico durante la etapa de diseño de un código paralelo. Ambos ámbitos descritos a continuación pueden implementarse de manera sincrónica o asincrónica.
  - **Punto a Punto:** Involucra dos tareas con una tarea actuando como el emisor/productor de datos, y la otra como el receptor/consumidor.
  - **Colectiva:** Involucra el intercambio de datos entre más de dos tareas, que a menudo se especifican como miembros de un grupo común o colectivo.
  
<p float="center">
  <img src="https://github.com/carlosalvarezh/EstructuraDatosAlgoritmos2/blob/main/images/Parallel45.PNG?raw=true" width="450" />
</p>

La imagen muestra cuatro patrones de comunicación comunes en la programación paralela y el procesamiento distribuido: broadcast, scatter, gather y reduction. Estos patrones son fundamentales en la implementación eficiente de algoritmos paralelos para la distribución y recopilación de datos, así como para la combinación de resultados.

- ***[Broadcast](https://www.cs.utexas.edu/users/AustinVilla/legged/papers/BroadcastProgrammingModels.pdf) (Difusión o Transmisión):*** Implica la distribución de datos desde un solo proceso (a menudo denominado el "root") a todos los demás procesos en un grupo. En un contexto de MPI (Message Passing Interface), por ejemplo, un valor o un conjunto de valores se envía desde un proceso fuente a todos los procesos dentro de un comunicador. Es útil cuando todos los procesos necesitan una copia idéntica de ciertos datos para realizar cálculos paralelos, como una constante necesaria para cálculos o una semilla para generadores de números aleatorios.

- ***[Scatter](https://www.gaurgaurav.com/patterns/scatter-gather/) (Dispersión o Distribución):*** Es un patrón donde el proceso root reparte segmentos diferentes y generalmente no superpuestos de un arreglo de datos a cada uno de los procesos en el grupo, incluido él mismo si es necesario. Este patrón es efectivo cuando se distribuye un arreglo grande entre varios procesos para su procesamiento paralelo, donde cada proceso trabaja en una parte del arreglo.

- ***Gather (Recolección o Agrupamiento):*** Es el proceso inverso a scatter. En lugar de distribuir datos, gather los colecta de todos los procesos en el grupo y los reúne en un único proceso root. Cada proceso envía su segmento de datos al proceso root, que luego los ensambla en un único conjunto de datos ordenado. Se utiliza a menudo al final de una operación paralela para recopilar resultados parciales de cada proceso y unirlos en un resultado final o para realizar un análisis posterior.

- ***[Reduction](https://en.wikipedia.org/wiki/Reduction_operator) (Reducción):*** Es un patrón de comunicación que combina los elementos de los datos de todos los procesos en el grupo utilizando una operación especificada (como sumar, maximizar, minimizar) y pasa el resultado combinado a todos los procesos o solo al proceso root. Se utiliza comúnmente para operaciones como calcular la suma de los elementos generados por cada proceso, encontrar el máximo o mínimo valor, o cualquier otra operación de reducción que necesite ser aplicada sobre los datos distribuidos entre los procesos.

En la imagen, los colores y las formas representan los datos específicos manejados por cada operación: broadcast muestra una pieza de información que se comparte desde un solo proceso a todos los demás; scatter muestra cómo se distribuye un conjunto de datos a diferentes procesos; gather representa la colecta de diferentes conjuntos de datos de vuelta al proceso root; y reduction ilustra cómo los datos de varios procesos se combinan en un solo resultado que puede ser repartido entre todos los procesos o acumulado en el proceso root.

Cada uno de estos patrones es esencial en situaciones donde la coordinación y la eficiencia en la manipulación de datos son críticas para el rendimiento y la exactitud de los cálculos paralelos.










**Eficiencia de las Comunicaciones**
- A menudo, el programador tiene opciones que pueden afectar el rendimiento de las comunicaciones. Elegir una plataforma con una red más rápida puede ser una opción.

**Sobrecarga y Complejidad**

<p float="center">
  <img src="https://github.com/carlosalvarezh/EstructuraDatosAlgoritmos2/blob/main/images/Parallel46.gif?raw=true" width="450" />
</p>

- Finalmente, ¡ten en cuenta que esta es solo una lista parcial de cosas a considerar!

Entender estos factores y cómo influyen en las comunicaciones dentro de un programa paralelo es crucial para el diseño eficiente de software que maximice el rendimiento mientras minimiza los cuellos de botella y la sobrecarga.

### Sincronización en Programación Paralela

La gestión de la secuencia de trabajo y de las tareas que la realizan es una consideración de diseño crítica para la mayoría de los programas paralelos.

- Puede ser un factor significativo en el rendimiento del programa (o la falta de él).
- A menudo requiere "serialización" de segmentos del programa.

<p float="center">
  <img src="https://github.com/carlosalvarezh/EstructuraDatosAlgoritmos2/blob/main/images/Parallel47.jpeg?raw=true" width="250" />
</p>

#### Tipos de Sincronización

**Barrera (Barrier)**
- Generalmente implica que todas las tareas están involucradas.
- Cada tarea ejecuta su trabajo hasta que alcanza la barrera. Entonces se detiene o "bloquea".
- Cuando la última tarea alcanza la barrera, todas las tareas se sincronizan.
- Lo que sucede a partir de aquí varía. A menudo, se debe hacer una sección de trabajo serial. En otros casos, las tareas se liberan automáticamente para continuar su trabajo.

**Bloqueo / Semáforo (Lock / Semaphore)**
- Puede involucrar a cualquier número de tareas.
- Típicamente usado para serializar (proteger) el acceso a datos globales o una sección de código. Solo una tarea a la vez puede usar (poseer) el bloqueo / semáforo / bandera.
- La primera tarea en adquirir el bloqueo lo "establece". Esta tarea puede entonces acceder de manera segura (serial) a los datos o código protegidos.
- Otras tareas pueden intentar adquirir el bloqueo pero deben esperar hasta que la tarea que posee el bloqueo lo libere.
- Puede ser bloqueante o no bloqueante.

**Operaciones de Comunicación Sincrónicas**
- Involucra solo a aquellas tareas que ejecutan una operación de comunicación.
- Cuando una tarea realiza una operación de comunicación, se requiere alguna forma de coordinación con las otras tarea(s) que participan en la comunicación. Por ejemplo, antes de que una tarea pueda realizar una operación de envío, primero debe recibir una confirmación de la tarea receptora de que está bien enviar.
- Discutido previamente en la sección de Comunicaciones.

***Consideraciones Adicionales***

- **Interbloqueo (Deadlock):** Ocurre cuando dos o más tareas esperan indefinidamente por recursos o eventos que están siendo bloqueados por las otras tareas.
- **Inanición (Starvation):** Una tarea nunca adquiere el acceso necesario a los recursos debido a que otros procesos están continuamente siendo preferidos.
- **Coordinación de Tareas:** Implica gestionar el orden y el tiempo en el que varias tareas acceden a recursos compartidos para evitar inconsistencias y errores.

La sincronización eficiente es crucial para asegurar la integridad de los datos y el comportamiento correcto de un programa paralelo. Sin embargo, también puede ser una fuente de reducción de rendimiento si no se implementa cuidadosamente, ya que puede aumentar el tiempo de inactividad de las tareas y limitar la concurrencia. La elección entre diferentes mecanismos de sincronización dependerá del problema específico, el modelo de programación paralela y las características de la arquitectura de hardware en uso.

### Dependencias de Datos

#### Introducción

Una dependencia existe entre instrucciones de un programa cuando el orden de ejecución de las instrucciones afecta los resultados del programa.
Una dependencia de datos resulta del uso múltiple de la misma ubicación(es) de almacenamiento por diferentes tareas.
Las dependencias son importantes en la programación paralela porque son uno de los principales inhibidores del paralelismo.

<p float="center">
  <img src="https://github.com/carlosalvarezh/EstructuraDatosAlgoritmos2/blob/main/images/Parallel48.jpeg?raw=true" width="200" />
</p>

Veamos algunos ejemplos:

**Dependencia de Datos Llevada por Bucle (Loop Carried Data Dependence):**
```fortran
DO J = MYSTART,MYEND
   A(J) = A(J-1) * 2.0
END DO
```
El valor de `A(J-1)` debe ser computado antes del valor de `A(J)`, por lo tanto, `A(J)` exhibe una dependencia de datos en `A(J-1)`. El paralelismo está inhibido.
Si la Tarea 2 tiene `A(J)` y la tarea 1 tiene `A(J-1)`, para calcular el valor correcto de `A(J)` es necesario:
- En arquitecturas de memoria distribuida - la tarea 2 debe obtener el valor de `A(J-1)` de la tarea 1 después de que esta termine su cálculo.
- En arquitecturas de memoria compartida - la tarea 2 debe leer `A(J-1)` después de que la tarea 1 lo actualice.

**Dependencia de Datos Independiente de Bucle (Loop Independent Data Dependence):**
```plaintext
tarea 1        tarea 2
------        ------
X = 2         X = 4
  .             .
  .             .
Y = X**2      Y = X**3
```
Al igual que con el ejemplo anterior, el paralelismo está inhibido. El valor de `Y` depende de:
- En arquitecturas de memoria distribuida - si o cuándo el valor de `X` se comunica entre las tareas.
- En arquitecturas de memoria compartida - qué tarea almacena el valor de `X` por última vez.

Aunque todas las dependencias de datos son importantes de identificar al diseñar programas paralelos, las dependencias llevadas por bucle son particularmente importantes ya que los bucles son posiblemente el objetivo más común de los esfuerzos de paralelización.

#### Cómo Manejar las Dependencias de Datos

- En arquitecturas de memoria distribuida - comunicar los datos requeridos en puntos de sincronización.
- En arquitecturas de memoria compartida - sincronizar las operaciones de lectura/escritura entre tareas.

Manejar las dependencias de datos requiere una comprensión profunda de cómo el flujo de datos y el control afectan el paralelismo potencial. Las optimizaciones pueden incluir reestructurar algoritmos para minimizar las dependencias o implementar mecanismos para gestionarlas de forma efectiva, lo que puede incluir sincronización cuidadosa y uso estratégico de recursos de memoria.

### Balanceo de Carga

<p float="center">
  <img src="https://github.com/carlosalvarezh/EstructuraDatosAlgoritmos2/blob/main/images/Parallel49.PNG?raw=true" width="750" />
</p>

El balanceo de carga se refiere a la práctica de distribuir cantidades aproximadamente iguales de trabajo entre las tareas para que todas estén ocupadas todo el tiempo. Puede considerarse una minimización del tiempo de inactividad de las tareas.

El balanceo de carga es importante para los programas paralelos por razones de rendimiento. Por ejemplo, si todas las tareas están sujetas a un punto de sincronización de barrera, la tarea más lenta determinará el rendimiento general.

#### Cómo Lograr el Balance de Carga

**Particionar Equitativamente el Trabajo que Recibe Cada Tarea**
- Para operaciones de arreglo/matriz donde cada tarea realiza un trabajo similar, distribuye de manera uniforme el conjunto de datos entre las tareas.
- Para iteraciones de bucle donde el trabajo realizado en cada iteración es similar, distribuye las iteraciones de manera uniforme entre las tareas.
- Si se utiliza una mezcla heterogénea de máquinas con características de rendimiento variables, asegúrate de usar alguna herramienta de análisis de rendimiento para detectar cualquier desequilibrio de carga y ajusta el trabajo en consecuencia.

**Usar Asignación Dinámica de Trabajo**

<p float="center">
  <img src="https://github.com/carlosalvarezh/EstructuraDatosAlgoritmos2/blob/main/images/Parallel50.PNG?raw=true" width="750" />
</p>

Ciertas clases de problemas resultan en desequilibrios de carga incluso si los datos están distribuidos uniformemente entre las tareas:

- Cuando la cantidad de trabajo que realizará cada tarea es intencionalmente variable, o no se puede predecir, puede ser útil utilizar un enfoque de grupo de tareas con programador. A medida que cada tarea termina su trabajo, recibe una nueva pieza de la cola de trabajo.

<p float="center">
  <img src="https://github.com/carlosalvarezh/EstructuraDatosAlgoritmos2/blob/main/images/Parallel51.gif?raw=true" width="400" />
</p>

Finalmente, puede ser necesario diseñar un algoritmo que detecte y maneje desequilibrios de carga a medida que ocurren dinámicamente dentro del código. Esto puede significar ajustes en tiempo real durante la ejecución del programa para reasignar el trabajo de las tareas más ocupadas a las menos ocupadas, a fin de optimizar el uso de los recursos y mejorar el rendimiento general.

### Granularidad

#### Razón de Cálculo / Comunicación

En la computación paralela, la granularidad es una medida cualitativa de la relación entre el cálculo y la comunicación.
Los períodos de cálculo están típicamente separados de los períodos de comunicación por eventos de sincronización.

#### Paralelismo de Grano Fino (Fine-grain Parallelism)

- Se realizan cantidades relativamente pequeñas de trabajo computacional entre eventos de comunicación.
- Baja relación de cálculo a comunicación.
- Facilita el balanceo de carga.
- Implica alta sobrecarga de comunicación y menos oportunidades para el aumento de rendimiento.
- Si la granularidad es demasiado fina, es posible que la sobrecarga requerida para las comunicaciones y sincronización entre tareas tome más tiempo que el cálculo.

<p float="center">
  <img src="https://github.com/carlosalvarezh/EstructuraDatosAlgoritmos2/blob/main/images/Parallel52.gif?raw=true" width="200" />
</p>

#### Paralelismo de Grano Grueso (Coarse-grain Parallelism)

- Se realizan cantidades relativamente grandes de trabajo computacional entre eventos de comunicación/sincronización.
- Alta relación de cálculo a comunicación.
- Implica más oportunidades para el aumento de rendimiento.
- Más difícil de balancear la carga de manera eficiente.

<p float="center">
  <img src="https://github.com/carlosalvarezh/EstructuraDatosAlgoritmos2/blob/main/images/Parallel53.gif?raw=true" width="200" />
</p>

#### ¿Cuál es Mejor?

La granularidad más eficiente depende del algoritmo y del entorno de hardware en el que se ejecuta.
- En la mayoría de los casos, la sobrecarga asociada con las comunicaciones y la sincronización es alta en relación con la velocidad de ejecución, por lo que es ventajoso tener una granularidad gruesa.
- El paralelismo de grano fino puede ayudar a reducir las sobrecargas debido al desequilibrio de carga.

En la práctica, la elección entre granularidad fina y gruesa a menudo requiere un equilibrio entre la sobrecarga de gestión de las tareas paralelas y la eficiencia con la que se pueden realizar los cálculos. Los algoritmos y aplicaciones específicos pueden requerir sintonía fina y ajustes iterativos para encontrar el nivel de granularidad óptimo que maximice el rendimiento dado un conjunto particular de restricciones de cómputo y comunicación.

### I/O en Computación Paralela

<p float="center">
  <img src="https://github.com/carlosalvarezh/EstructuraDatosAlgoritmos2/blob/main/images/Parallel54.gif?raw=true" width="500" />
</p>

#### Las Malas Noticias

**Jerarquía de Memoria y Operaciones de I/O**
- Las operaciones de I/O (entrada/salida) son generalmente consideradas como inhibidores del paralelismo.
- Las operaciones de I/O requieren órdenes de magnitud más tiempo que las operaciones de memoria.
- Los sistemas de I/O paralelo pueden ser inmaduros o no estar disponibles para todas las plataformas.
- En un entorno donde todas las tareas ven el mismo espacio de archivo, las operaciones de escritura pueden resultar en la sobreescritura de archivos.
- Las operaciones de lectura pueden verse afectadas por la capacidad del servidor de archivos para manejar múltiples solicitudes de lectura al mismo tiempo.
- I/O que debe realizarse a través de la red (NFS, no local) puede causar cuellos de botella severos e incluso colapsar servidores de archivos.

#### Las Buenas Noticias

**Sistemas de Archivos Paralelos**
- Están disponibles sistemas de archivos paralelos. Por ejemplo:
  - GPFS: General Parallel File System (IBM), ahora llamado IBM Spectrum Scale.
  - Lustre: para clústeres de Linux (Intel)
  - HDFS: Hadoop Distributed File System (Apache)
  - PanFS: Panasas ActiveScale File System para clústeres de Linux (Panasas, Inc.)
  - Y más - ver [Lista de sistemas de archivos paralelos distribuidos en Wikipedia](http://en.wikipedia.org/wiki/List_of_file_systems#Distributed_parallel_file_systems)

La especificación de la interfaz de programación de I/O paralelo para MPI ha estado disponible desde 1996 como parte de MPI-2. Las implementaciones de proveedores y gratuitas ahora son comúnmente disponibles.

#### Algunas Sugerencias

- **Regla #1:** Reducir el I/O total tanto como sea posible.
- Si tienes acceso a un sistema de archivos paralelo, úsalo.
- Escribir grandes bloques de datos en lugar de pequeños suele ser significativamente más eficiente.
- Un menor número de archivos grandes rinde mejor que muchos archivos pequeños.
- Confinar las operaciones de I/O a porciones seriales específicas del trabajo, y luego usar comunicaciones paralelas para distribuir datos a tareas paralelas. Por ejemplo, la Tarea 1 podría leer un archivo de entrada y luego comunicar los datos necesarios a otras tareas. Del mismo modo, la Tarea 1 podría realizar operaciones de escritura después de recibir los datos necesarios de todas las demás tareas.
- Agregar operaciones de I/O entre tareas: en lugar de que muchas tareas realicen I/O, tener un subconjunto de tareas que lo hagan.

En resumen, una estrategia eficaz de I/O en entornos de computación paralela puede tener un impacto significativo en el rendimiento general de una aplicación. Optimizar las operaciones de I/O para reducir la sobrecarga y evitar cuellos de botella es esencial para aprovechar al máximo los recursos de computación paralela.

### Depuración (Debugging)

#### Introducción

<p float="center">
  <img src="https://github.com/carlosalvarezh/EstructuraDatosAlgoritmos2/blob/main/images/Parallel55.gif?raw=true" width="750" />
</p>

La depuración de códigos paralelos puede ser increíblemente difícil, particularmente a medida que los códigos se escalan.

La buena noticia es que hay excelentes depuradores disponibles para ayudar:
- **Hilos (Threaded):** pthreads y OpenMP
- **MPI**
- **GPU / aceleradores**
- **Híbrido**

Los usuarios de Livermore Computing tienen acceso a varias herramientas de depuración paralela instaladas en los clústeres de LC:
- **TotalView** de RogueWave Software
- **DDT** de Allinea
- **Inspector** de Intel
- **Stack Trace Analysis Tool (STAT):** desarrollado localmente en LLNL

Todas estas herramientas tienen una curva de aprendizaje asociada con ellas.

Para detalles e información para comenzar, ver:
- Páginas web de LC en [hpc.llnl.gov/software/development-environment-software](https://hpc.llnl.gov/software/development-environment-software)
- Tutorial de TotalView: [hpc.llnl.gov/documentation/tutorials/totalview-tutorial](https://hpc.llnl.gov/documentation/tutorials/totalview-tutorial)

#### Consejos para la Depuración de Códigos Paralelos

1. **Comenzar Pequeño:** Comienza con una versión reducida del problema para simplificar la detección de errores.
2. **Aumentar Gradualmente:** Aumenta el tamaño y la complejidad del código de forma gradual, depurando a cada paso.
3. **Utilizar Asertos:** Usa asertos y cheques de coherencia de datos para validar el estado del programa en puntos críticos.
4. **Comunicación y Sincronización:** Presta especial atención a la comunicación y sincronización entre tareas, ya que estos son puntos comunes donde pueden surgir problemas.
5. **Utilizar Herramientas Especializadas:** Aprovecha las herramientas de depuración paralela, que pueden ofrecer capacidades de detección de patrones de bloqueo, carreras de datos y otros problemas de concurrencia.
6. **Registros Verbosos:** Mantén registros detallados de las ejecuciones del programa para rastrear y reproducir errores.
7. **Pruebas Unitarias y de Integración:** Implementa pruebas unitarias y de integración para verificar el funcionamiento correcto de componentes individuales y su integración.

La depuración en el contexto de la computación paralela es a menudo más arte que ciencia, y requiere una combinación de herramientas adecuadas, técnicas sistemáticas y a veces, una buena dosis de paciencia.

### Análisis de Rendimiento y Ajuste

<p float="center">
  <img src="https://github.com/carlosalvarezh/EstructuraDatosAlgoritmos2/blob/main/images/Parallel56.jpeg?raw=true" width="500" />
</p>

Al igual que con la depuración, analizar y ajustar el rendimiento de programas paralelos puede ser mucho más desafiante que para programas seriales.

Afortunadamente, existen varias herramientas excelentes para el análisis de rendimiento y el ajuste de programas paralelos.

Los usuarios de Livermore Computing tienen acceso a varias de estas herramientas, la mayoría de las cuales están disponibles en todos los clústeres de producción.

Algunos puntos de partida para herramientas instaladas en sistemas de LC:
- Páginas web de LC: [hpc.llnl.gov/software/development-environment-software](https://hpc.llnl.gov/software/development-environment-software)
- TAU (Tuning and Analysis Utilities): [Página de TAU](http://www.cs.uoregon.edu/research/tau/docs.php)
- HPCToolkit: [Documentación de HPCToolkit](http://hpctoolkit.org/documentation.html)
- Open|Speedshop: [Sitio de Open|Speedshop](https://www.openspeedshop.org/)
- Vampir / Vampirtrace: [Sitio de Vampir](http://vampir.eu/)
- Valgrind: [Sitio de Valgrind](http://valgrind.org/)
- PAPI (Performance Application Programming Interface): [Sitio de PAPI](http://icl.cs.utk.edu/papi/)
- mpiP: [Sitio de mpiP](http://mpip.sourceforge.net/)
- memP: [Sitio de memP](http://memp.sourceforge.net/)

***Consejos para el Análisis de Rendimiento y Ajuste:***

1. **Identificar Cuellos de Botella:** Usa herramientas de perfilado para identificar dónde pasa la mayor parte del tiempo tu programa. Estos son a menudo los mejores candidatos para la optimización.

2. **Evaluar la Paralelización:** Verifica que la carga de trabajo esté bien balanceada entre las tareas y que la sincronización y la comunicación no estén impactando negativamente el rendimiento.

3. **Medir el Impacto de la Sincronización y la Comunicación:** Analiza cuánto tiempo se gasta en barreras de sincronización y en comunicación de datos entre procesos.

4. **Optimizar el Uso de Memoria:** Asegúrate de que tu programa está utilizando la memoria de manera eficiente para evitar la contención y para que el acceso a la memoria no sea un cuello de botella.

5. **Ajustar la Granularidad:** Si es necesario, ajusta la granularidad del paralelismo para mejorar el balance de carga y reducir la sobrecarga de comunicación.

6. **Usar Contadores de Hardware:** Utiliza PAPI u otras interfaces para acceder a contadores de hardware y entender mejor el comportamiento de tu aplicación en relación con el hardware subyacente.

7. **Revisar Algoritmos y Estructuras de Datos:** Considera la posibilidad de cambiar algoritmos y estructuras de datos por otros más eficientes o más adecuados para la computación paralela.

8. **Iterar el Proceso:** El ajuste del rendimiento es un proceso iterativo. A menudo, una optimización llevará a la identificación de la siguiente área de mejora potencial.

Analizar y ajustar el rendimiento es esencial para obtener el máximo provecho de los sistemas de cómputo paralelo. La integración de un análisis de rendimiento detallado y continuo en el ciclo de vida del desarrollo del software puede conducir a mejoras significativas en el rendimiento y la escalabilidad de los programas paralelos.